### Bibliotecas utilizadas em todas as celulas

In [1]:
import numpy as np
import talib as ta

from bot.historico_precos import get_price_history
from bot.models.coin_pair import CoinPair
from db.duckdb_csv import save_to_csv_duckdb
from segredos import BASE_DIR


### Definição da única moeda utilizada

In [2]:
coinpair = CoinPair(
    base='BTC',
    quote='BRL',
)

### Primeiro dataset:
 - Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
 - Saida: preço de fechamento deslocado em uma linha

In [3]:
df = get_price_history(coin_pair=coinpair)
df['y'] = df['close'].shift(-1)
df = df.dropna()
save_to_csv_duckdb(
    df,
    BASE_DIR
    + '/datasets'
    + '/saida_como_preco_do_ativo_'
    + f'{coinpair.bitpreco_websocket}_{coinpair.exchange.value}'
    + '.csv',
)

True

### Segundo dataset:
- Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
- Saida: 5 níveis de tendência geral

    -2: Baixa forte

    -1: Tendência de baixa

    0: Tendência neutra

    1: Tendência de alta

    2: Alta forte

In [4]:
df = get_price_history(coin_pair=coinpair)
df = df.copy()
# Converter Series para numpy arrays
close_arr = df['close'].to_numpy(dtype=float)
high_arr = df['high'].to_numpy(dtype=float)
low_arr = df['low'].to_numpy(dtype=float)
volume_arr = df['volume'].to_numpy(dtype=float)

##### Calcular os indicadores osciladores

In [5]:
def AO(df):
    """
    Awesome Oscillator
    """
    df['media'] = (df['open'] + df['close']) / 2
    df['media'] = df['media'].to_numpy(dtype=float)

    # Definir os períodos para as médias móveis
    periodo_curto = 5
    periodo_longo = 34

    # Calcular as médias móveis usando TA-Lib
    df['SMA_Curta'] = ta.SMA(df['media'].values, timeperiod=periodo_curto)
    df['SMA_Longa'] = ta.SMA(df['media'].values, timeperiod=periodo_longo)

    return df['SMA_Curta'] - df['SMA_Longa']

In [6]:
# Índice de Força Relativa (14)
df['RSI'] = ta.RSI(close_arr, timeperiod=14)
# Estocástico %K (14, 3, 3)
df['stoch_k'], df['stoch_d'] = ta.STOCH(
    high_arr,
    low_arr,
    close_arr,
    fastk_period=14,
    slowk_period=3,
    slowd_period=3,
)
# índice Canal de Commodities
df['CCI'] = ta.CCI(high_arr, low_arr, close_arr, timeperiod=20)
# Índice Direcional Médio
df['ADX'] = ta.ADX(high_arr, low_arr, close_arr, timeperiod=14)
# Oscilador Maravilhoso
df['AO'] = AO(df)
# Momentum (10)
df['MOM'] = ta.MOM(close_arr, timeperiod=10)
# Nível MACD (12,26)
df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(
    close_arr,
    fastperiod=12,
    slowperiod=26,
    signalperiod=9,
)
# IFR Estocástico Rápido (3, 3, 14, 14)
df['stoch_rsi_k'], df['stoch_rsi_d'] = ta.STOCHF(
    high=high_arr,
    low=low_arr,
    close=close_arr,
    fastk_period=14,
    fastd_period=3,
    fastd_matype=0,
)
# Range Percentual de Williams (14)
df['WILLR'] = ta.WILLR(high_arr, low_arr, close_arr, timeperiod=14)
# Bull Bear Power
df['BBP'] = ta.CDLKICKINGBYLENGTH(
    open=df['open'].to_numpy(dtype=float),
    high=high_arr,
    low=low_arr,
    close=close_arr,
)
# Ultimate Oscillator (7, 14, 28)
df['ULTOSC'] = ta.ULTOSC(
    high_arr,
    low_arr,
    close_arr,
    timeperiod1=7,
    timeperiod2=14,
    timeperiod3=28,
)

##### Calcular indicadores médias móveis

In [7]:
# Calcular a VWMA
def calcular_vwma(df, periodo):
    # Calcular a soma ponderada dos preços
    soma_ponderada = df['close'] * df['volume']

    # Calcular a soma dos volumes
    soma_volumes = df['volume']

    # Calcular a VWMA
    vwma = (
        soma_ponderada.rolling(window=periodo).sum()
        / soma_volumes.rolling(window=periodo).sum()
    )

    return vwma

In [8]:
# Calcular a Linha Base (Kijun-sen)
def calcular_kijun_sen(df):
    maximos = df['high'].rolling(window=26).max()
    minimos = df['low'].rolling(window=26).min()
    kijun_sen = (maximos + minimos) / 2
    return kijun_sen

In [9]:
# Calcular a Média Móvel de Hull
def calcular_hma(df, n):
    df['close'] = df['close'].to_numpy(dtype=float)
    wma_metade = ta.WMA(df['close'].values, timeperiod=int(n / 2))
    wma_completa = ta.WMA(df['close'].values, timeperiod=n)
    hma_intermediaria = 2 * wma_metade - wma_completa
    hma = ta.WMA(hma_intermediaria, timeperiod=int(np.sqrt(n)))
    return hma

In [10]:
# Média Movel Exponencial (10)
df['EMA10'] = ta.EMA(close_arr, timeperiod=10)
# Média Movel Simples (10)
df['SMA10'] = ta.SMA(close_arr, timeperiod=10)
# Média Movel Exponencial (20)
df['EMA20'] = ta.EMA(close_arr, timeperiod=20)
# Média Movel Simples (20)
df['SMA20'] = ta.SMA(close_arr, timeperiod=20)
# Média Movel Exponencial (30)
df['EMA30'] = ta.EMA(close_arr, timeperiod=30)
# Média Movel Simples (30)
df['SMA30'] = ta.SMA(close_arr, timeperiod=30)
# Média Movel Exponencial (50)
df['EMA50'] = ta.EMA(close_arr, timeperiod=50)
# Média Movel Simples (50)
df['SMA50'] = ta.SMA(close_arr, timeperiod=50)
# Média Movel Exponencial (100)
df['EMA100'] = ta.EMA(close_arr, timeperiod=100)
# Média Movel Simples (100)
df['SMA100'] = ta.SMA(close_arr, timeperiod=100)
# Média Movel Exponencial (200)
df['EMA200'] = ta.EMA(close_arr, timeperiod=200)
# Média Movel Simples (200)
df['SMA200'] = ta.SMA(close_arr, timeperiod=200)
# Linha Base Ichimoku (9, 26, 52, 26)
df['Kijun-sen'] = calcular_kijun_sen(df)
# Média Móvel Ponderada pelo Volume (20)
df['VWMA'] = calcular_vwma(df, 20)
# Média Móvel de Hull (9)
df['HMA'] = calcular_hma(df, 9)

##### Calcular as tendências individuais de cada indicador

In [11]:
# Função para classificar tendências em 5 níveis
def classificar_tendencia(valor, limites):
    """
    Classifica um valor em 5 níveis de tendência:
    -2: Baixa forte
    -1: Tendência de baixa
    0: Tendência neutra
    1: Tendência de alta
    2: Alta forte
    """
    forte_baixa, baixa, alta, forte_alta = limites

    if valor <= forte_baixa:
        return -2  # Baixa forte
    elif valor <= baixa:
        return -1  # Tendência de baixa
    elif valor >= forte_alta:
        return 2  # Alta forte
    elif valor >= alta:
        return 1  # Tendência de alta
    else:
        return 0  # Tendência neutra


# Classifica osciladores
# RSI (0-100)
df['tendencia_RSI'] = df['RSI'].apply(
    lambda x: classificar_tendencia(x, (30, 40, 60, 70))
)

# Estocástico %K (0-100)
df['tendencia_stoch_k'] = df['stoch_k'].apply(
    lambda x: classificar_tendencia(x, (20, 35, 65, 80))
)

# Estocástico %D (0-100)
df['tendencia_stoch_d'] = df['stoch_d'].apply(
    lambda x: classificar_tendencia(x, (20, 35, 65, 80))
)

# CCI (-100 a +100 tipicamente)
df['tendencia_CCI'] = df['CCI'].apply(
    lambda x: classificar_tendencia(x, (-200, -100, 100, 200))
)

# ADX (0-100)
df['tendencia_ADX'] = df['ADX'].apply(
    lambda x: 2 if x > 50 else (1 if x > 25 else 0)
)

# Oscilador Maravilhoso (AO)
# Criando coluna com o valor anterior para comparação
df['AO_anterior'] = df['AO'].shift(1)
df['tendencia_AO'] = df.apply(
    lambda row: 2
    if row['AO'] > 0 and row['AO'] > row['AO_anterior']
    else (
        1
        if row['AO'] > 0
        else (
            -2
            if row['AO'] < 0 and row['AO'] < row['AO_anterior']
            else (-1 if row['AO'] < 0 else 0)
        )
    ),
    axis=1,
)

# Momentum
df['tendencia_MOM'] = df.apply(
    lambda row: 2
    if row['MOM'] > 3
    else (
        1
        if row['MOM'] > 0
        else (-2 if row['MOM'] < -3 else (-1 if row['MOM'] < 0 else 0))
    ),
    axis=1,
)

# MACD
# Criando coluna com o valor anterior para comparação
df['MACD_hist_anterior'] = df['MACD_hist'].shift(1)
df['tendencia_MACD'] = df.apply(
    lambda row: 2
    if row['MACD'] > row['MACD_signal']
    and row['MACD_hist'] > row['MACD_hist_anterior']
    else (
        1
        if row['MACD'] > row['MACD_signal']
        else (
            -2
            if row['MACD'] < row['MACD_signal']
            and row['MACD_hist'] < row['MACD_hist_anterior']
            else (-1 if row['MACD'] < row['MACD_signal'] else 0)
        )
    ),
    axis=1,
)

# IFR Estocástico Rápido
df['tendencia_stoch_rsi_k'] = df['stoch_rsi_k'].apply(
    lambda x: classificar_tendencia(x, (20, 35, 65, 80))
)
df['tendencia_stoch_rsi_d'] = df['stoch_rsi_d'].apply(
    lambda x: classificar_tendencia(x, (20, 35, 65, 80))
)

# Williams %R (0 a -100)
df['tendencia_WILLR'] = df['WILLR'].apply(
    lambda x: classificar_tendencia(-x, (-80, -65, -35, -20))
)

# Bull Bear Power
df['tendencia_BBP'] = df['BBP'].apply(
    lambda x: 2
    if x > 75
    else (1 if x > 0 else (-2 if x < -75 else (-1 if x < 0 else 0)))
)

# Ultimate Oscillator (0-100)
df['tendencia_ULTOSC'] = df['ULTOSC'].apply(
    lambda x: classificar_tendencia(x, (30, 40, 60, 70))
)

# Classificar tendências de médias móveis
# Para todas as médias móveis, compara o preço atual (close) com a média
df['tendencia_EMA10'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA10'] * 1.03
    else (
        1
        if row['close'] > row['EMA10']
        else (
            -2
            if row['close'] < row['EMA10'] * 0.97
            else (-1 if row['close'] < row['EMA10'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA10'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA10'] * 1.03
    else (
        1
        if row['close'] > row['SMA10']
        else (
            -2
            if row['close'] < row['SMA10'] * 0.97
            else (-1 if row['close'] < row['SMA10'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_EMA20'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA20'] * 1.05
    else (
        1
        if row['close'] > row['EMA20']
        else (
            -2
            if row['close'] < row['EMA20'] * 0.95
            else (-1 if row['close'] < row['EMA20'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA20'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA20'] * 1.05
    else (
        1
        if row['close'] > row['SMA20']
        else (
            -2
            if row['close'] < row['SMA20'] * 0.95
            else (-1 if row['close'] < row['SMA20'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_EMA30'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA30'] * 1.07
    else (
        1
        if row['close'] > row['EMA30']
        else (
            -2
            if row['close'] < row['EMA30'] * 0.93
            else (-1 if row['close'] < row['EMA30'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA30'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA30'] * 1.07
    else (
        1
        if row['close'] > row['SMA30']
        else (
            -2
            if row['close'] < row['SMA30'] * 0.93
            else (-1 if row['close'] < row['SMA30'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_EMA50'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA50'] * 1.1
    else (
        1
        if row['close'] > row['EMA50']
        else (
            -2
            if row['close'] < row['EMA50'] * 0.9
            else (-1 if row['close'] < row['EMA50'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA50'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA50'] * 1.1
    else (
        1
        if row['close'] > row['SMA50']
        else (
            -2
            if row['close'] < row['SMA50'] * 0.9
            else (-1 if row['close'] < row['SMA50'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_EMA100'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA100'] * 1.15
    else (
        1
        if row['close'] > row['EMA100']
        else (
            -2
            if row['close'] < row['EMA100'] * 0.85
            else (-1 if row['close'] < row['EMA100'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA100'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA100'] * 1.15
    else (
        1
        if row['close'] > row['SMA100']
        else (
            -2
            if row['close'] < row['SMA100'] * 0.85
            else (-1 if row['close'] < row['SMA100'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_EMA200'] = df.apply(
    lambda row: 2
    if row['close'] > row['EMA200'] * 1.2
    else (
        1
        if row['close'] > row['EMA200']
        else (
            -2
            if row['close'] < row['EMA200'] * 0.8
            else (-1 if row['close'] < row['EMA200'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_SMA200'] = df.apply(
    lambda row: 2
    if row['close'] > row['SMA200'] * 1.2
    else (
        1
        if row['close'] > row['SMA200']
        else (
            -2
            if row['close'] < row['SMA200'] * 0.8
            else (-1 if row['close'] < row['SMA200'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_Kijun'] = df.apply(
    lambda row: 2
    if row['close'] > row['Kijun-sen'] * 1.05
    else (
        1
        if row['close'] > row['Kijun-sen']
        else (
            -2
            if row['close'] < row['Kijun-sen'] * 0.95
            else (-1 if row['close'] < row['Kijun-sen'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_VWMA'] = df.apply(
    lambda row: 2
    if row['close'] > row['VWMA'] * 1.05
    else (
        1
        if row['close'] > row['VWMA']
        else (
            -2
            if row['close'] < row['VWMA'] * 0.95
            else (-1 if row['close'] < row['VWMA'] else 0)
        )
    ),
    axis=1,
)

df['tendencia_HMA'] = df.apply(
    lambda row: 2
    if row['close'] > row['HMA'] * 1.03
    else (
        1
        if row['close'] > row['HMA']
        else (
            -2
            if row['close'] < row['HMA'] * 0.97
            else (-1 if row['close'] < row['HMA'] else 0)
        )
    ),
    axis=1,
)

##### Fazer a moda das tendências para a tendência geral

In [12]:
# Selecionar todas as colunas que começam com 'tendencia_'
colunas_tendencia = [col for col in df.columns if col.startswith('tendencia_')]

print(f'Total de indicadores de tendência: {len(colunas_tendencia)}')
print(f"Indicadores utilizados: {', '.join(colunas_tendencia)}")

# classificar tendência geral vendo o que a maioria
# dos indicadores estão indicando
df['tendencia_geral'] = df[colunas_tendencia].mode(axis=1)[0]

Total de indicadores de tendência: 28
Indicadores utilizados: tendencia_RSI, tendencia_stoch_k, tendencia_stoch_d, tendencia_CCI, tendencia_ADX, tendencia_AO, tendencia_MOM, tendencia_MACD, tendencia_stoch_rsi_k, tendencia_stoch_rsi_d, tendencia_WILLR, tendencia_BBP, tendencia_ULTOSC, tendencia_EMA10, tendencia_SMA10, tendencia_EMA20, tendencia_SMA20, tendencia_EMA30, tendencia_SMA30, tendencia_EMA50, tendencia_SMA50, tendencia_EMA100, tendencia_SMA100, tendencia_EMA200, tendencia_SMA200, tendencia_Kijun, tendencia_VWMA, tendencia_HMA


##### fazer a saída que é a tendência geral deslocada para o minuto seguinte

In [14]:
df['y'] = df['tendencia_geral'].shift(-1)
# dropar as linhas que não tem o valor de y
df = df.dropna()
# Selecionar apenas as colunas volume, open, high, low, close e y
df = df[
    [
        'timestamp',
        'volume',
        'open',
        'high',
        'low',
        'close',
        'y',
    ]
]
# Salvar o dataframe em um arquivo CSV com o duckdb por que são muitos dados
save_to_csv_duckdb(
    df,
    BASE_DIR
    + '/datasets'
    + '/saida_como_tendencia_geral_'
    + f'{coinpair.bitpreco_websocket}_{coinpair.exchange.value}'
    + '.csv',
)

True

### Terceiro dataset:
- Entradas: volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo
- Saida: sinal de compra(-1), venda(1) ou neutro, ou seja não comprar e nem vender (0)